In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# # Load the Iris dataset
# iris = load_iris()
# X_train, y_train = iris.data, iris.target


In [5]:
# Load data
train_data = pd.read_csv('/content/drive/MyDrive/data4you/W2_D5/W2_D5_notebooks/training_validation/training_dataset.csv')
# validation_data = pd.read_csv('/content/drive/MyDrive/data4you/W2_D5/W2_D5_notebooks/training_validation/validation_dataset.csv')

# Remove rows with NaN values
train_data = train_data.dropna()
# validation_data = validation_data.dropna()


# Separate features and target for training data
X_train = train_data.drop(columns=['TARGET'])  # Replace 'target' with the name of your target column
y_train = train_data['TARGET']

# # Encode target labels (for multiclass)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

print(X_train)
print(y_train)

[[ 0.58491275  0.71025156  1.01257936 ... -0.42060838  0.6168163
   0.7262174 ]
 [ 0.08593813 -0.6062293   0.04928391 ...  2.55477445  0.23758602
  -0.47718306]
 [ 1.05745396  0.33510808  0.43195341 ... -0.55362574 -0.25689138
  -0.88695481]
 ...
 [-0.99000906 -1.14755063 -0.91707687 ... -0.11987362  0.99251389
  -0.64752057]
 [-1.45175452 -1.90427474 -1.23298279 ... -0.23704861 -0.06071956
  -0.45966859]
 [-1.1752242  -1.48001035 -1.20201634 ... -0.61991149 -0.13982662
  -0.83834735]]
[0 0 0 ... 4 4 4]


In [6]:

# Random Hill Climbing for hyperparameter tuning
def random_hill_climbing_svm(X, y, step_size=0.1, iterations=50):
    best_score = 0
    best_params = {'C': 1, 'gamma': 0.1}
    scores = []

    for i in range(iterations):
        # Random tweaks to the current best parameters
        C = best_params['C'] + np.random.uniform(-step_size, step_size)
        gamma = best_params['gamma'] + np.random.uniform(-step_size, step_size)

        # Ensure C and gamma are within valid ranges
        C = max(0.1, min(100, C))
        gamma = max(0.0001, min(1, gamma))

        # Train SVM model with these hyperparameters
        model = SVC(C=C, gamma=gamma)
        score = np.mean(cross_val_score(model, X, y, cv=5, scoring='accuracy'))
        scores.append((C, gamma, score))

        # Update best score and parameters if current score is better
        if score > best_score:
            best_score = score
            best_params = {'C': C, 'gamma': gamma}




    return best_params, best_score, scores

# Perform random hill climbing
best_params, best_score, all_scores = random_hill_climbing_svm(X_train, y_train)

best_params, best_score

({'C': 0.9926708094691828, 'gamma': 0.00941526510177966}, 0.9363004003202562)

In [ ]:




# Define the parameter ranges for tuning
param_ranges = {
    "max_depth": (1, 25),  # Depth of the tree
    "min_samples_split": (2, 20),  # Minimum samples to split a node
    "min_samples_leaf": (1, 20)  # Minimum samples in a leaf node
}


# Function to evaluate a model's performance
def evaluate_model(params):
    """Train and evaluate the model with given hyperparameters."""
    model = DecisionTreeClassifier(
        max_depth=params["max_depth"],
        min_samples_split=params["min_samples_split"],
        min_samples_leaf=params["min_samples_leaf"],
        random_state=42
    )
    # Perform cross-validation and return the negative mean accuracy (for minimization)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    return -scores.mean()


# Random Hill Climbing for hyperparameter tuning
def random_hill_climbing(param_ranges, max_iter=100, step_size=1):
    """Optimize hyperparameters using Random Hill Climbing."""
    # Initialize a random solution within parameter ranges
    # Create a random number generator with a fixed seed (42) for reproducibility.
    # This ensures that the random numbers generated are the same every time the code is run.
    rng = np.random.default_rng(42)

    # Initialize a dictionary `current_params` with random values for each hyperparameter.
    # For each key (hyperparameter name) in `param_ranges`, a random integer is generated
    # within the specified range [low, high] (inclusive of low and high).
    current_params = {
        key: rng.integers(low, high + 1)  # Generate random integer in [low, high]
        for key, (low, high) in param_ranges.items()  # Iterate over the hyperparameter ranges
    }


    current_score = evaluate_model(current_params)

    best_params = current_params
    best_score = current_score

    for _ in range(max_iter):
        # Generate a random neighbor
        neighbor_params = current_params.copy()
        for key in param_ranges:
            low, high = param_ranges[key]
            neighbor_params[key] += np.random.randint(-step_size, step_size + 1)  # high is exclusive, either -1 0 1 are generated
            neighbor_params[key] = np.clip(neighbor_params[key], low, high)

        # Evaluate the neighbor
        neighbor_score = evaluate_model(neighbor_params)
        print(f"For params {neighbor_params} my score is {neighbor_score}")
        # Accept neighbor if better
        if neighbor_score < current_score:
            current_params = neighbor_params
            current_score = neighbor_score

            # Update the best solution
            if neighbor_score < best_score:
                best_params = neighbor_params
                best_score = neighbor_score

    return best_params, -best_score  # Return accuracy as a positive value


# Perform hyperparameter tuning
best_params, best_accuracy = random_hill_climbing(param_ranges, max_iter=100)

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)
